In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
rnn_model = load_model(r"D:\Project_Env\News Article classification\News Article classification\Save_model\news_classification_model_rnn.h5")
lstm_model=load_model(r"D:\Project_Env\News Article classification\News Article classification\Save_model\News_classNews_classification_model_LSTM_1.h5")

In [2]:
LENGTH_THRESHOLD = 20  # Number of tokens (words)

In [3]:
import gensim

# Load pre-trained embeddings (ensure the correct file path)
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(r"D:\Project_Env\News Article classification\News Article classification\data\numberbatch-en.txt", binary=False)

In [4]:
import numpy as np

def get_embedding_vector(text, word_vectors, embedding_dim=300, max_length=200):
    tokens = text.split()
    embeddings = [word_vectors[word] for word in tokens if word in word_vectors]

    if len(embeddings) == 0:
        return np.zeros((max_length, embedding_dim), dtype=np.float32)

    embeddings = np.array(embeddings[:max_length], dtype=np.float32)
    padding_needed = max_length - len(embeddings)

    if padding_needed > 0:
        embeddings = np.vstack([embeddings, np.zeros((padding_needed, embedding_dim), dtype=np.float32)])

    return embeddings


In [5]:
# Define file paths
SAVE_TRAIN_PATH = r"D:\Project_Env\News Article classification\News Article classification\data\train.pkl"
SAVE_TEST_PATH = r"D:\Project_Env\News Article classification\News Article classification\data\test.pkl"

import pickle

# Load training data
with open(SAVE_TRAIN_PATH, "rb") as f:
    train_data = pickle.load(f)

# Load test data
with open(SAVE_TEST_PATH, "rb") as f:
    test_data = pickle.load(f)

print("Data successfully loaded!")
print(train_data.keys())
print(test_data.keys())


Data successfully loaded!
Index(['class_index', 'title', 'description', 'text', 'clean_text', 'vector'], dtype='object')
Index(['class_index', 'title', 'description', 'text', 'clean_text', 'vector'], dtype='object')


In [6]:
# Check unique class indexes in the dataset
unique_classes = train_data["class_index"].unique()
print("Unique class indexes:", unique_classes)

Unique class indexes: [3 4 2 1]


In [7]:
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
print(label_map)

{0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load pre-trained model
model = load_model(r"D:\Project_Env\News Article classification\News Article classification\Save_model\News_classification_model_LSTM_1.h5")

# Mapping class indexes to category names
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

while True:
    # Take user input
    new_text = input("\nEnter a news article (or type 'exit' to quit): ")
    
    # Exit condition
    if new_text.lower() == 'exit':
        print("Exiting...")
        break

    # Convert text to embedding
    X_new = np.array([get_embedding_vector(new_text, word_vectors)], dtype=np.float32)

    # Predict class probabilities
    predictions = model.predict(X_new)

    # Predict class index
    predicted_class_index = np.argmax(predictions, axis=1)[0]  # Extract single value

    # Get class name
    predicted_class_name = label_map[predicted_class_index]

    print(f"Predicted class: {predicted_class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
Predicted class: World
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Predicted class: Business
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Predicted class: Sci/Tech
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted class: Sports
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Predicted class: Sci/Tech
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
Predicted class: Sci/Tech
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted class: Sci/Tech
Exiting...


In [9]:
# from tensorflow.keras.models import load_model
import numpy as np

# # Load both models
# rnn_model = load_model(r"C:\Users\Trisha\jupytor tutorial\Final-Year-Project\News Article classification\saved_model\news_classification_model_rnn.h5")
# lstm_model = load_model(r"C:\Users\Trisha\jupytor tutorial\Final-Year-Project\News Article classification\saved_model\news_classification_model.h5")

# # Category index mapping
# class_labels = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

# # Define threshold
# LENGTH_THRESHOLD = 50

# Prediction function
def hybrid_predict(text, word_vectors, rnn_model, lstm_model, threshold=50):
    tokens = text.split()
    embedding = get_embedding_vector(text, word_vectors)
    input_data = np.expand_dims(embedding, axis=0)

    if len(tokens) <= threshold:
        prediction = rnn_model.predict(input_data)
        model_used = "RNN"
    else:
        prediction = lstm_model.predict(input_data)
        model_used = "LSTM"

    predicted_class = np.argmax(prediction)
    return label_map[predicted_class], model_used


In [10]:
while True:
    # Take user input
    new_text = input("\nEnter a news article (or type 'exit' to quit): ")
    
    # Exit condition
    if new_text.lower() == 'exit':
        print("Exiting...")
        break

    # Check input length
    tokens = new_text.split()
    X_new = np.array([get_embedding_vector(new_text, word_vectors)], dtype=np.float32)

    # Choose model based on input length
    if len(tokens) <= 20:
        predictions = rnn_model.predict(X_new)
        model_used = "RNN"
    else:
        predictions = lstm_model.predict(X_new)
        model_used = "LSTM"

    # Predict class
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = label_map[predicted_class_index]

    print(f"Predicted class: {predicted_class_name} ({model_used})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: Business (RNN)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted class: World (RNN)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: Sports (LSTM)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class: Sports (LSTM)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Predicted class: Sports (RNN)
Exiting...
